<a href="https://colab.research.google.com/github/ElizabethGarrison/Working-With-Data-Fundamentals/blob/main/Copy_of_LinearModels2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Models, part II: Logistic Regression

We'll use the global social indicators data to develop a logistic regression model and pratice interpreting the results.

First, we'll import the libraries we'll need for this model.


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sb
import math


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Next, we'll read in the data sources and create the DataFrame with all of our variables.

In [ ]:
GlobalIndicators1 = pd.read_excel('http://data.shortell.nyc/files/HumanDevelopment.xlsx', index_col='Country', na_values=['NA'])
GlobalIndicators1.head()

,HDI Rank,Human Development Index (HDI),Life Expectancy at Birth,Expected Years of Education,Mean Years of Education,Gross National Income (GNI) per Capita,GNI per Capita Rank Minus HDI Rank
Country,,,,,,,
Norway,1,0.944,81.6,17.5,12.6,64992,5
Australia,2,0.935,82.4,20.2,13.0,42261,17
Switzerland,3,0.930,83.0,15.8,12.8,56431,6
Denmark,4,0.923,80.2,18.7,12.7,44025,11
Netherlands,5,0.922,81.6,17.9,11.9,45435,9


In [ ]:
GlobalIndicators2 = pd.read_excel('http://data.shortell.nyc/files/GenderDevelopment.xlsx', index_col='Country', na_values=['NA'])
GlobalIndicators2.head()

,GDI Rank,Gender Development Index (GDI),Human Development Index (Female),Human Development Index (Male),Life Expectancy at Birth (Female),Life Expectancy at Birth (Male),Expected Years of Education (Female),Expected Years of Education (Male),Mean Years of Education (Female),Mean Years of Education (Male),Estimated Gross National Income per Capita (Female),Estimated Gross National Income per Capita (Male)
Country,,,,,,,,,,,,
Norway,1,0.996,0.940,0.944,83.6,79.5,18.2,16.8,12.7,12.5,57140.0,72825.0
Australia,2,0.976,0.922,0.945,84.5,80.3,20.7,19.7,13.1,12.9,33688.0,50914.0
Switzerland,3,0.950,0.898,0.945,85.0,80.8,15.7,15.9,11.5,13.1,44132.0,69077.0
Denmark,4,0.977,0.912,0.934,82.2,78.3,19.3,18.1,12.8,12.7,36439.0,51727.0
Netherlands,5,0.947,0.893,0.943,83.3,79.7,18.0,17.9,11.6,12.2,29500.0,61641.0


In [ ]:
GlobalIndicators3 = pd.read_excel('http://data.shortell.nyc/files/GenderInequality.xlsx', index_col='Country', na_values=['NA'])
GlobalIndicators3.head()

,GII Rank,Gender Inequality Index (GII),Maternal Mortality Ratio,Adolescent Birth Rate,Percent Representation in Parliament,Population with Secondary Education (Female),Population with Secondary Education (Male),Labour Force Participation Rate (Female),Labour Force Participation Rate (Male)
Country,,,,,,,,,
Norway,1,0.067,4.0,7.8,39.6,97.4,96.7,61.2,68.7
Australia,2,0.110,6.0,12.1,30.5,94.3,94.6,58.8,71.8
Switzerland,3,0.028,6.0,1.9,28.5,95.0,96.6,61.8,74.9
Denmark,4,0.048,5.0,5.1,38.0,95.5,96.6,58.7,66.4
Netherlands,5,0.062,6.0,6.2,36.9,87.7,90.5,58.5,70.6


In [ ]:
GlobalIndicatorsTotal = pd.concat([GlobalIndicators1, GlobalIndicators2, GlobalIndicators3], axis=1)
GlobalIndicatorsTotal.shape

(188, 28)

Now, we'll compute a binary variable that will be our dependent variable, Y. Then, we'll identify the relevant independent variables and put them in a new DataFrame, X. At that point, we can compute the model.

In [ ]:
GlobalIndicatorsTotal['Gender Inequality Index (GII)'].describe()

count    155.000000
mean       0.365884
std        0.191457
min        0.016000
25%        0.184000
50%        0.385000
75%        0.524500
max        0.744000
Name: Gender Inequality Index (GII), dtype: float64

In [ ]:
GlobalIndicatorsTotal['GII Binary'] = 0
GlobalIndicatorsTotal.loc[GlobalIndicatorsTotal['Gender Inequality Index (GII)'] < 0.19, ['GII Binary']] = 1 #These are nations with a low gender inequality score--that is, the highest gender equality
GlobalIndicatorsTotal['GII Binary'].describe()

count    188.000000
mean       0.207447
std        0.406561
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000
Name: GII Binary, dtype: float64

In [ ]:
GlobalIndicatorsTotal['GII Binary']

Country
Norway                      1
Australia                   1
Switzerland                 1
Denmark                     1
Netherlands                 1
                           ..
Burundi                     0
Chad                        0
Eritrea                     0
Central African Republic    0
Niger                       0
Name: GII Binary, Length: 188, dtype: int64

In [ ]:
Y = GlobalIndicatorsTotal['GII Binary']
X = GlobalIndicatorsTotal[['Percent Representation in Parliament', 'Population with Secondary Education (Female)', 'Labour Force Participation Rate (Female)', 'Life Expectancy at Birth', 'Gross National Income (GNI) per Capita']]
model0 = sm.Logit(Y, X, missing='drop').fit()
print(model0.summary())

Optimization terminated successfully.
         Current function value: 0.321133
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:             GII Binary   No. Observations:                  156
Model:                          Logit   Df Residuals:                      151
Method:                           MLE   Df Model:                            4
Date:                Thu, 02 Apr 2020   Pseudo R-squ.:                  0.4289
Time:                        14:14:10   Log-Likelihood:                -50.097
converged:                       True   LL-Null:                       -87.724
Covariance Type:            nonrobust   LLR p-value:                 1.760e-15
                                                   coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Percent Representation in Parliament    

In [ ]:
print(math.exp(model0.params[0]), math.exp(model0.params[1]), 1/math.exp(model0.params[2]), 1/math.exp(model0.params[3]), math.exp(1000*model0.params[4]))
# We need to exponentiate (or the take anti-logs of) the coefficients in order to interpret them as odds. For the negative coefficients, it is useful to take the inverse of the result
# and interpret it in the opposite direction (that is, the odds of not being in the high gender equality group). You can also change the increment of change in X
# as is the case here with the parameter for GNI per capita; I changed the increment to $1000 instead of $1.

1.113047941683358 1.0677546839634509 1.0600888084476103 1.086297381643493 1.0540509355600345


In [ ]:
model0_marginals = model0.get_margeff()
print(model0_marginals.summary())

        Logit Marginal Effects       
Dep. Variable:             GII Binary
Method:                          dydx
At:                           overall
                                                  dy/dx    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Percent Representation in Parliament             0.0109      0.002      4.922      0.000       0.007       0.015
Population with Secondary Education (Female)     0.0066      0.001      5.889      0.000       0.004       0.009
Labour Force Participation Rate (Female)        -0.0059      0.002     -2.653      0.008      -0.010      -0.002
Life Expectancy at Birth                        -0.0084      0.001     -6.576      0.000      -0.011      -0.006
Gross National Income (GNI) per Capita        5.335e-06   1.13e-06      4.728      0.000    3.12e-06    7.55e-06


In [ ]:
model0_pred = model0.pred_table()
print(model0_pred) # Correct predictions are on the diagonal of the 2d array.

[[109.   8.]
 [ 12.  27.]]


In [ ]:
correct_i = 109 / (109 + 8) # The proportion of correct predictions of 0.
correct_j = 27 / (27 + 12) # The proportion of correct predictions of 1.
print(correct_i, correct_j)

0.9316239316239316 0.6923076923076923


## Activity

1. Find and read into a DataFrame a suitable dataset. You may use the global social indictors data from the example here. You may need to combine files, as shown here.

2. Identify a dependent variable to explain. Create a binary variable of this measure, if needed. Explain why you chose this variable or recoded in the way you did.

3. Build a model to explain the DV. You can use the odds (anti-log of the coefficients) or the marginal effects to test for the unique effects of each predictor.

4. Explain the results.

In [11]:
import pandas as pd

# Read the dataset from the given URL and set 'Country' as the index column
GlobalIndicators3 = pd.read_excel('http://data.shortell.nyc/files/GenderInequality.xlsx', index_col='Country', na_values=['NA'])
GlobalIndicators3.head()

,GII Rank,Gender Inequality Index (GII),Maternal Mortality Ratio,Adolescent Birth Rate,Percent Representation in Parliament,Population with Secondary Education (Female),Population with Secondary Education (Male),Labour Force Participation Rate (Female),Labour Force Participation Rate (Male)
Country,,,,,,,,,
Norway,1,0.067,4.0,7.8,39.6,97.4,96.7,61.2,68.7
Australia,2,0.110,6.0,12.1,30.5,94.3,94.6,58.8,71.8
Switzerland,3,0.028,6.0,1.9,28.5,95.0,96.6,61.8,74.9
Denmark,4,0.048,5.0,5.1,38.0,95.5,96.6,58.7,66.4
Netherlands,5,0.062,6.0,6.2,36.9,87.7,90.5,58.5,70.6


In [12]:
# Define a threshold to categorize countries into High/Low Gender Inequality
threshold = 0.5  # You can adjust this threshold as needed

# Create a binary variable 'HighGenderInequality' based on the threshold
GlobalIndicators3['HighGenderInequality'] = (GlobalIndicators3['Gender Inequality Index (GII)'] > threshold).astype(int)


This code creates a new binary variable 'HighGenderInequality' where countries with GII greater than the threshold are labeled as 1 (High Gender Inequality) and others as 0 (Low Gender Inequality).



In [14]:
import statsmodels.api as sm

# Remove rows with missing values
GlobalIndicators3_cleaned = GlobalIndicators3.dropna()

# Define the predictors and add a constant for the intercept
X = GlobalIndicators3_cleaned[['Maternal Mortality Ratio', 'Adolescent Birth Rate', 'Percent Representation in Parliament', 'Population with Secondary Education (Female)', 'Population with Secondary Education (Male)', 'Labour Force Participation Rate (Female)', 'Labour Force Participation Rate (Male)']]
X = sm.add_constant(X)

# Define the dependent variable
y = GlobalIndicators3_cleaned['HighGenderInequality']

# Fit a logistic regression model
model = sm.Logit(y, X).fit()

# Print model summary
print(model.summary())



Optimization terminated successfully.
         Current function value: 0.102028
         Iterations 11
                            Logit Regression Results                            
Dep. Variable:     HighGenderInequality   No. Observations:                  155
Model:                            Logit   Df Residuals:                      147
Method:                             MLE   Df Model:                            7
Date:                  Mon, 02 Oct 2023   Pseudo R-squ.:                  0.8306
Time:                          23:28:52   Log-Likelihood:                -15.814
converged:                         True   LL-Null:                       -93.378
Covariance Type:              nonrobust   LLR p-value:                 3.396e-30
                                                   coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
const                  

The logistic regression model successfully converged, and its findings show a significant explanatory power with a pseudo-R-squared value of 0.8306. The coefficients of the predictors shed light on the elements which contributes to high gender inequality. Notably, the 'Maternal Mortality Ratio,' 'Adolescent Birth Rate,' and 'Labour Force Participation Rate (Male)' all have positive coefficients, indicating that higher maternal mortality, adolescent birth rates, and male labor force participation rates are associated with a higher likelihood of high gender inequality. Alternatively, the coefficient for "Percent Representation in Parliament" is negative, indicating that there is a correlation between reduced gender disparity and more representation of women in the legislature. Less significant relationships seen for the remaining variables, including "Population with Secondary Education." It is important to note that the model's pseudo R-squared indicates that a sizable percentage of the variability in high gender inequality is still unaccounted for, and the possibility of quasi-separation reveals that some parameters may not be found due to the features of the data. To fully comprehend the many processes causing gender imbalance, more research and model improvement may be required.